In [ ]:
import pandas as pd
import numpy as np

# Load the merged data
merged_csv_path = '../data/processed/train_merged.csv'
df = pd.read_csv(merged_csv_path, low_memory=False, parse_dates=['Date'])

print('Data loaded successfully.')
df.head()

### Zaman Bazlı Ayırma (Time-Based Split)

In [ ]:
# Son 6 haftayı validasyon seti olarak ayıralım
validation_date = df['Date'].max() - pd.DateOffset(weeks=6)
train_df = df[df['Date'] < validation_date]
val_df = df[df['Date'] >= validation_date]

print('Training set shape:', train_df.shape)
print('Validation set shape:', val_df.shape)

### Baseline Model: Mağaza Ortalaması

In [ ]:
# Eğitim setindeki her mağazanın ortalama satışını hesapla
store_avg_sales = train_df.groupby('Store')['Sales'].mean().reset_index()
store_avg_sales = store_avg_sales.rename(columns={'Sales': 'AvgSales'})

print('Store average sales calculated.')
store_avg_sales.head()

In [ ]:
# Validasyon setine ortalama satışları ekle
val_df_merged = pd.merge(val_df, store_avg_sales, on='Store', how='left')

# Tahminleri (predictions) ata
val_df_merged['Predictions'] = val_df_merged['AvgSales']

# Eğer bir mağaza train setinde yoksa (yeni mağaza), ortalamayı tüm mağazaların ortalaması ile doldur
global_avg_sales = train_df['Sales'].mean()
val_df_merged['Predictions'].fillna(global_avg_sales, inplace=True)

print('Predictions made on the validation set.')
val_df_merged[['Date', 'Store', 'Sales', 'Predictions']].head()

### Performans Değerlendirme: RMSPE

In [ ]:
def rmsp_error(y_true, y_pred):
    # Satışların 0 olduğu durumları hesaba katma
    y_true = y_true[y_pred != 0]
    y_pred = y_pred[y_pred != 0]
    
    # RMSPE hesaplama
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Sadece mağazanın açık olduğu ve satışın pozitif olduğu günleri değerlendir
val_filtered = val_df_merged[(val_df_merged['Open'] == 1) & (val_df_merged['Sales'] > 0)]

rmspe = rmsp_error(val_filtered['Sales'], val_filtered['Predictions'])
print(f'Baseline Model RMSPE: {rmspe:.4f}')